#### **Решить систему дифференциальных уравнений**

In [15]:
import numpy as np
import pandas as pd

Выпишем условия данных функций для работы и последующей проверки:

In [16]:
def dx_dt(x, y):
    return -2 * x + 4 * y

def dy_dt(x, y):
    return -x + 3 * y
    
def check_x(N, a=0, b=1):
    return [4 * np.exp(-t) - np.exp(2 * t) for t in np.linspace(a, b, N + 1)]

def check_y(N, a=0, b=1):
    return [np.exp(-t) - np.exp(2 * t) for t in np.linspace(a, b, N+1)]

Метод Рунге-Кутты четвёртого порядка:

In [27]:
def rk4(N, a=0, b=1, x0=3, y0=0):
    h = (b - a) / N
    x = np.zeros(N + 1)
    x[0] = x0
    y = np.zeros(N + 1)
    y[0] = y0

    for i in range(1, N + 1):
        kx0 = dx_dt(x[i - 1], y[i - 1])
        ky0 = dy_dt(x[i - 1], y[i - 1])
        kx1 = dx_dt(x[i - 1] + kx0 * h / 2, y[i - 1] + ky0 * h / 2)
        ky1 = dy_dt(x[i - 1] + kx0 * h / 2, y[i - 1] + ky0 * h / 2)
        kx2 = dx_dt(x[i - 1] + kx1 * h / 2, y[i - 1] + ky1 * h / 2)
        ky2 = dy_dt(x[i - 1] + kx1 * h / 2, y[i - 1] + ky1 * h/ 2)
        kx3 = dx_dt(x[i - 1] + kx2 * h, y[i - 1] + h * ky2)
        ky3 = dy_dt(x[i - 1] + kx2 * h, y[i - 1] + h * ky2)
        x[i] = x[i - 1] + h / 6 * (kx0 + 2 * kx1 + 2 * kx2 + kx3)
        y[i] = y[i - 1] + h / 6 * (ky0 + 2 * ky1 + 2 * ky2 + ky3)

    return x, y

Cхема Адамса–Бэшфорта–Моултона требует работы с несколькими базовыми точками - мы их найдём с помощью метода Рунге-Кутты:

In [28]:
def ABM(N, a=0, b=1, x0=3, y0=0):
    h = (b - a) / N
    x = np.zeros(N + 1)
    y = np.zeros(N + 1)
    RKX, RKY = rk4(3, a, h*3)

    for i in range(0, 4):
        x[i], y[i] = RKX[i], RKY[i]
    for i in range(3, N):
        bx = x[i] + h/24 * (55 * dx_dt(x[i], y[i]) - 59 * dx_dt(x[i-1], y[i-1]) + 37 * dx_dt(x[i-2], y[i-2]) - 9 * dx_dt(x[i-3], y[i-3]))
        by = y[i] + h/24 * (55 * dy_dt(x[i], y[i]) - 59 * dy_dt(x[i-1], y[i-1]) + 37 * dy_dt(x[i-2], y[i-2]) - 9 * dy_dt(x[i-3], y[i-3]))
        x[i+1] = x[i] + h/24 * (9 * dx_dt(bx, by) + 19 * dx_dt(x[i], y[i]) - 5 * dx_dt(x[i-1], y[i-1]) + dx_dt(x[i-2], y[i-2]))
        y[i+1] = y[i] + h/24 * (9 * dy_dt(bx, by) + 19 * dy_dt(x[i], y[i]) - 5 * dy_dt(x[i-1], y[i-1]) + dy_dt(x[i-2], y[i-2]))
        
    return x, y

Создадим таблицу для проверки качества метода:

In [29]:
data = pd.DataFrame(np.array([
                            [max( abs(rk4(i)[0][-1] - check_x(i)[-1]), 
                                  abs(rk4(i)[1][-1] - check_y(i)[-1]) ) for i in [2 ** j for j in range(2, 9)]],
                            [np.log2(
                             max( abs(rk4(i)[0][-1] - check_x(i)[-1]), 
                                  abs(rk4(i)[1][-1] - check_y(i)[-1]) ) /
                             max( abs(rk4(i * 2)[0][-1] - check_x(i * 2)[-1]), 
                                  abs(rk4(i * 2)[1][-1] - check_y(i * 2)[-1]) )
                                ) for i in [2 ** j for j in range(2, 9)]], 
                                
                            [max(abs(ABM(i)[0][-1] - check_x(i)[-1]), 
                                 abs(ABM(i)[1][-1] - check_y(i)[-1])) for i in [2 ** j for j in range(2, 9)]], 
                            [np.log2(
                             max( abs(ABM(i)[0][-1] - check_x(i)[-1]), 
                                  abs(ABM(i)[1][-1] - check_y(i)[-1])) / 
                             max( abs(ABM(i * 2)[0][-1] - check_x(i * 2)[-1]), 
                                  abs(ABM(i * 2)[1][-1] - check_y(i * 2)[-1]) )
                                ) for i in [2 ** j for j in range(2, 9)]]]).T, 
                    columns=['eps_rk', 'P_rk', 'eps_ABM', 'P_ABM'], 
                    index=[2 ** i for i in range(2, 9)])
data

,eps_rk,P_rk,eps_ABM,P_ABM
4,5.144808e-03,3.706307,6.207307e-03,5.243563
8,3.941484e-04,3.852127,1.638451e-04,2.724273
16,2.729316e-05,3.925809,2.479393e-05,2.863576
32,1.795840e-06,3.962841,3.406616e-06,3.568288
64,1.151685e-07,3.981405,2.871847e-07,3.803852
128,7.291408e-09,3.990696,2.056306e-08,3.905911
256,4.586616e-10,3.995512,1.371801e-09,3.953934
